# 0.0 Imports

In [ ]:
import pandas as pd
import inflection
import itertools
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.core.display import HTML

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

import xgboost as xgb
from xgboost import XGBRegressor

## 0.1 Helper functions

In [ ]:
#função para melhorar a exibição dos gráficos

def jupyter_settings():
  %matplotlib inline
  %pylab inline
  plt.style.use( 'bmh' )
  plt.rcParams['figure.figsize'] = [25, 12]
  plt.rcParams['font.size'] = 24
  display( HTML( '<style>.container { width:100% !important; }</style>') )
  pd.options.display.max_columns = None
  pd.options.display.max_rows = None
  pd.set_option( 'display.expand_frame_repr', False )
  sns.set()
    


In [ ]:
jupyter_settings()

## 0.2 Importando os dados

Nessa seção iremos importar os dados para o notebook, e juntar as tabelas de features, store e treino, para ter todos os dados no mesmo lugar e conseguir visualizar melhor a relação entre as variáveis. 

In [ ]:
df_stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')

In [ ]:
df_stores.head()

In [ ]:
df_features = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')

In [ ]:
df_features.head()

In [ ]:
feature_store = pd.merge( df_features, df_stores, how='left', on='Store' )

In [ ]:
feature_store.tail(5)

Dados de treino:

In [ ]:
df_train = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')

In [ ]:
# range da data dos dados de treino: 05-02-2010 até 26-10-2012
df_train.head()

In [ ]:
df_raw_train = pd.merge( df_train, feature_store, how='inner', on=['Store','Date','IsHoliday'] )

In [ ]:
df_raw_train.tail(5)

Dados de teste:

In [ ]:
df_test = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')
df_test.tail()

In [ ]:
df_raw_test = pd.merge( df_test, feature_store, how='inner', on=['Store','Date','IsHoliday'] )

In [ ]:
df_raw_test.tail()

# 1.0 Descrição dos Dados

Conhecimento dos nossos dados: número de linhas, colunas, tipos dos dados e ajustes na forma de como vieram escritos.

In [ ]:
df1 = df_raw_train.copy()

In [ ]:
df1.head()

In [ ]:
# padronizando os nomes das colunas
cols_old = ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
'Fuel_Price', 'MarkDown1', 'MarkDown2',
'MarkDown3', 'MarkDown4', 'MarkDown5',
'CPI','Unemployment', 'Type', 'Size']
snakecase = lambda x: inflection.underscore( x )
cols_new = list( map( snakecase, cols_old ) )
# rename
df1.columns = cols_new

In [ ]:
df1.head()

In [ ]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

In [ ]:
df1.dtypes

## 1.1 Tratamento dos Dados

Iremos transformar as variáveis para ficar mais simples de trabalhar com elas.

In [ ]:
#conversão do tipo da feature 'date'
df1['date'] = pd.to_datetime( df1['date'] )

In [ ]:
# conversão do tipo da feature 'is_holiday'
#1 = é feriado, 0 = não é feriado 
df1['is_holiday'] = df1['is_holiday'].astype('int64')

In [ ]:
df1.dtypes

## 1.2 Checkout NA

Verificação de valores nulos para seu devido tratamento.

In [ ]:
df1.isna().sum()

## 1.3 Fillout NA

Seção destinada ao preenchimento dos dados faltantes.

Como os dados das promoções estão faltantes, na seção 4.0 (Análise exploratório dos dados) iremos verificar estatisticamente se eles tem grande impacto na variável resposta, para determinar se devemos trabalhar com eles, ou se podemos retirá-los.

In [ ]:
# Variáveis a serem analisadas.

#1 - mark_down1
#2 - mark_down2
#3 - mark_down3
#4 - mark_down4
#5 - mark_down5

## 1.4 Estatística Descritiva

Será criado um dataframe para features numéricas e categóricas, para identificarmos como ela se comportam e suas principais características estatísticas.

In [ ]:
#dataframe com os atributos numéricos
num_attributes = df1.select_dtypes( include= ['int64', 'float64'])
num_attributes.head()

In [ ]:
cat_attributes = df1.select_dtypes( exclude= ['int64', 'float64', 'datetime64[ns]'])
cat_attributes.head()

### 1.4.1 Atributos Numéricos

In [ ]:
# Tendência central: Média e Mediana
ct1 = pd.DataFrame(num_attributes.apply(np.mean) ).T

ct2 = pd.DataFrame(num_attributes.apply(np.median) ).T

In [ ]:
# Dispersão: std, min, max, range, skew, kurtosis

d1 = pd.DataFrame(num_attributes.apply(np.std) ).T
d2 = pd.DataFrame(num_attributes.apply(min ) ).T
d3 = pd.DataFrame(num_attributes.apply(max ) ).T
#Para fazer o range: max - min
## como eu preciso fazer uma função que passar por todos os items do dataframe
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

In [ ]:
#concatenando 
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std',
             'skew', 'kurtosis']

In [ ]:
m

A partir dessas informações, podemos verificar os seguintes pontos em algumas features:

- [weekly_sales] - temos lojas marcando vendas negativas (necessário verificar).
                 - a média é o dobro da mediana, evidenciando a presença de outliers.
                 
                 
- [temperature] A temperatura pode parecer estranhar, porém para a escala dos Estados Unidos (F°), se encontra em normalidade.


### 1.5.2 Atributos Categóricos

In [ ]:
cat_attributes.apply( lambda x: x.unique().shape[0] )

In [ ]:
#boxplot da variável categórica 'type' com relação as vendas semanais

# o 'weekly_sales' foi setado em 50000 para poder visualizar melhor os dados, devido a outliers
aux = df1[(df1['weekly_sales'] > 0) & (df1['weekly_sales'] < 50000)]

sns.boxplot( x='type', y='weekly_sales', data=aux )

Podemos verificar que as lojas tipo 'A' são as que mais vendem.

# 2.0 Feature Engineering

In [ ]:
df2 = df1.copy()
df2.head()

## 2.1 Feature engineering

A partir da feature [date], podemos extrair outras features que podem nos ajudar a entender melhor o fenômeno.

In [ ]:
########### Dates Features ########################

# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.weekofyear

# year week
df2['year_week'] = df2['date'].dt.strftime( '%Y-%W' )

# 3.0 Filtragem das Variáveis

In [ ]:
df3 = df2.copy()
df3.head()

## 3.1 Filtragem das Linhas

Como queremos prever as vendas semanais, pra cada departamento pra cada loja,  irei somente considerar as linhas aonde tenho uma weekly_sales >= 0.

In [ ]:
df3 = df3[(df3['weekly_sales'] >= 0)]
df3.head()

# 4.0 Análise Exploratória dos Dados (EDA)

Aqui na análise exploratória dos dados, podemos ganhar experiência no negócio, avaliando como cada feature se comporta, podemos validar as hipóteses de negócio e perceber quais variáveis são importantes para o modelo.

In [ ]:
df4 = df3.copy()
df4.head()

## 4.1 Análise Univariada

Aqui verificamos como cada feature se comporta.

### 4.1.1 Variável Resposta

In [ ]:
sns.distplot( df4['weekly_sales'], kde=False );

Temos uma alta concentração de vendas para valores menores, que vai caindo conforme as quantias são maiores.

### 4.1.2 Variáveis Numéricas

In [ ]:
num_attributes.hist(bins = 25);

### 4.1.3 Variáveis Categóricas

In [ ]:
plt.subplot( 3, 2, 1)
sns.countplot(df4['type']);

#dataframe auxiliar para visualizar a distribuição sem os outliers
aux = df4[(df4['weekly_sales'] > 0) & (df4['weekly_sales'] < 50000)]

plt.subplot( 3, 2, 2)
sns.kdeplot( aux[aux['type'] == 'A']['weekly_sales'], label = 'A', shade=True )
sns.kdeplot( aux[aux['type'] == 'B']['weekly_sales'], label = 'B', shade=True )
sns.kdeplot( aux[aux['type'] == 'C']['weekly_sales'], label = 'C', shade=True )

## 4.2 Análise Bivariada

Nesta seção vamos verificar com cada variável impacta na variável resposta.

O gráfico abaixo mostra a média de vendas dos anos de 2010,2011 e 2012. Como podemos observar, elas são muito parecidas.

In [ ]:
#média das vendas por ano
weekly_sales_2010 = df4[df4.year == 2010].groupby('week_of_year')['weekly_sales'].mean()
weekly_sales_2011 = df4[df4.year == 2011].groupby('week_of_year')['weekly_sales'].mean()
weekly_sales_2012 = df4[df4.year == 2012].groupby('week_of_year')['weekly_sales'].mean()

plt.figure(figsize=(22,8))
plt.plot(weekly_sales_2010.index, weekly_sales_2010.values, 's-b')
plt.plot(weekly_sales_2011.index, weekly_sales_2011.values, 'o-r')
plt.plot(weekly_sales_2012.index, weekly_sales_2012.values, '*-g')
plt.xticks( rotation=90 );

plt.title("Venda Média Semanal - Por Ano", fontsize=24)
plt.legend(['2010', '2011', '2012'], fontsize=20);

### 4.2.1 Análise das Séries Temporais das lojas

Iremos analisar as vendas de uma loja como exemplo ao longo do tempo, para tentar entender seu comportamento.

Loja 10

In [ ]:
df4.set_index('date', inplace=True)

loja10 = df4[df4.store == 10]

In [ ]:
vendas10 = pd.DataFrame(loja10.weekly_sales.groupby(loja10.index).sum())

vendas10.reset_index(inplace = True)

vendas10['date'] = pd.to_datetime(vendas10['date'])

vendas10.set_index('date',inplace = True)

In [ ]:
vendas10.weekly_sales.plot(figsize=(10,6), title= 'Vendas Semanais - Loja 10', fontsize=14, color = 'salmon')
plt.show()

Iremos decompor a série temporal em 3 componentes: tendência, sazonalidade e resíduo.

In [ ]:
decomposition = seasonal_decompose(vendas10.weekly_sales, period=12)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(12, 10)
plt.show();

Podemos observar pelos gráficos:

    - As claras formações de tendências ao longo do ano e o spike quando as vendas aumentam no final de cada ano.
    - Uma sazonalidade muito regular.
    - Resíduos que se distoam justamente nos pontos de pico de vendas.

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 5)

# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, d and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in list(itertools.product(p, d, q))]

In [ ]:
mod = sm.tsa.statespace.SARIMAX(vendas10,
                                order=(4, 4, 3),
                                seasonal_order=(1, 1, 0, 52),   
                                enforce_invertibility=False)

results = mod.fit()

print(results.summary().tables[1])

In [ ]:
plt.style.use('seaborn-pastel')
results.plot_diagnostics(figsize=(15, 12))
plt.show()

Com base nos nossos resultados, vamos verificar se conseguimos prever as vendas para os ultimos 90 dias

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2012-07-27'), dynamic=False)
pred_ci = pred.conf_int()

In [ ]:
ax = vendas10['2010':].plot(label='Original')
pred.predicted_mean.plot(ax=ax, label='One Step Ahead Forecast', alpha=.7)

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='r', alpha=.2)

ax.set_xlabel('Período')
ax.set_ylabel('Weekly Sales')
plt.legend()

plt.show()

Como podemos observar na figura (região em vermelho), ainda temos alguns picos em pontos que não condizem em como as vendas se comportaram.

Vamos tentar agora uma predição dinâmica.

In [ ]:
pred_dynamic = results.get_prediction(start=pd.to_datetime('2012-7-27'), dynamic=True, full_results=True)
pred_dynamic_ci = pred_dynamic.conf_int()

In [ ]:
ax = vendas10['2010':].plot(label='Observado', figsize=(12, 8))
pred_dynamic.predicted_mean.plot(label='Dynamic Forecast', ax=ax)

ax.fill_between(pred_dynamic_ci.index,
                pred_dynamic_ci.iloc[:, 0],
                pred_dynamic_ci.iloc[:, 1], color='r', alpha=.25)

ax.fill_betweenx(ax.get_ylim(), pd.to_datetime('2012-7-26'), vendas10.index[-1],
                 alpha=.1, zorder=-1)

ax.set_xlabel('Período ')
ax.set_ylabel('Weekly Sales')

plt.legend()
plt.show()


Agora na predição dinâmica, podemos observar as vendas andando mais próximas.

In [ ]:
df4.reset_index(inplace = True)

In [ ]:
df4 = df3.copy()

### H1. Quanto maior o CPI, menor as vendas.

Falso: Temos alto número de vendas com CPI baixo e CPI alto. 
Porém, estatisticamente, ambas possuem uma alta correlação inversa (-0.82): quanto maior o CPI, menor as vendas.

In [ ]:
aux1 = df4[['cpi', 'weekly_sales']].groupby( 'cpi' ).sum().reset_index()

plt.subplot(1,2,1)
# criação de uma lista aonde eu posso organizar os valores
bins = list( np.arange( 110, 227, 10) )
# pd.cut: coloca a coluna selecionada dentro do bin que eu criei
aux1['cpi_binned'] = pd.cut( aux1['cpi'],bins=bins )
aux2 = aux1[['cpi_binned', 'weekly_sales']].groupby('cpi_binned' ).sum().reset_index()
sns.barplot( x='cpi_binned', y='weekly_sales', data=aux2 );
plt.xticks( rotation=90 );

plt.subplot(1,2,2)
x = sns.heatmap( aux1.corr( method='pearson' ), annot=True );
bottom, top = x.get_ylim()
x.set_ylim( bottom+0.5, top-0.5 );

In [ ]:
weekly_sales_2010 = df4[df4.year == 2010].groupby('week_of_year')['weekly_sales'].mean()
cpi_2010          = df4[df4.year == 2010].groupby('week_of_year')['cpi'].mean()

weekly_sales_2011 = df4[df4.year == 2011].groupby('week_of_year')['weekly_sales'].mean()
cpi_2011          = df4[df4.year == 2011].groupby('week_of_year')['cpi'].mean()

plt.subplot(2,2,1)
plt.title("Média das Vendas Semanais - 2010")
plt.plot(weekly_sales_2010.index, weekly_sales_2010.values, 's-b')

plt.subplot(2,2,3)
plt.title("CPI Médio - 2010")
plt.plot(cpi_2010.index, cpi_2010.values, 'o-r')

plt.subplot(2,2,2)
plt.title("Média das Vendas Semanais - 2011")
plt.plot(weekly_sales_2011.index, weekly_sales_2011.values, 's-b')

plt.subplot(2,2,4)
plt.title("CPI Médio - 2011")
plt.plot(cpi_2011.index, cpi_2011.values, 'o-r')

Podemos observar nos gráficos acima:

 - O CPI médio de 2010 praticamente não mudou (variação entre 167 e 168.4)
 - O CPI médio de 2011 subiu quase 6 pontos
 
 Mesmo com essa variação não linear dos CPI's, as vendas médias semanais dos 2 anos ainda seguem o mesmo padrão.
 
 O CPI possui uma alta correlação inversa com as vendas semanais.

### H2. Quanto maior o Unemployment , menor as vendas.

Falso: a taxa de desemprego não afeta as vendas.

In [ ]:
aux1 = df4[['unemployment', 'weekly_sales']].groupby( 'unemployment' ).sum().reset_index()

plt.subplot(1,2,1)
bins = list( np.arange( 3, 15, 0.1) )
aux1['unemployment_binned'] = pd.cut( aux1['unemployment'],bins=bins )
aux2 = aux1[['unemployment_binned', 'weekly_sales']].groupby('unemployment_binned' ).sum().reset_index()
sns.barplot( x='unemployment_binned', y='weekly_sales', data=aux2 );
plt.xticks( rotation=90 );

plt.subplot(1,2,2)
x = sns.heatmap( aux1.corr( method='pearson' ), annot=True );
bottom, top = x.get_ylim()
x.set_ylim( bottom+0.5, top-0.5 );

In [ ]:
weekly_sales_2010 = df4[df4.year == 2010].groupby('week_of_year')['weekly_sales'].mean()
unemployment_2010          = df4[df4.year == 2010].groupby('week_of_year')['unemployment'].mean()

weekly_sales_2011 = df4[df4.year == 2011].groupby('week_of_year')['weekly_sales'].mean()
unemployment_2011          = df4[df4.year == 2011].groupby('week_of_year')['unemployment'].mean()

plt.subplot(2,2,1)
plt.title("Média das Vendas Semanais - 2010")
plt.plot(weekly_sales_2010.index, weekly_sales_2010.values, 's-b')

plt.subplot(2,2,3)
plt.title("Taxa Média de Desemprego - 2010")
plt.plot(unemployment_2010.index, unemployment_2010.values, 'o-r')

plt.subplot(2,2,2)
plt.title("Média das Vendas Semanais - 2011")
plt.plot(weekly_sales_2011.index, weekly_sales_2011.values, 's-b')

plt.subplot(2,2,4)
plt.title("Taxa Média de Desemprego - 2011")
plt.plot(unemployment_2011.index, unemployment_2011.values, 'o-r')

Nos gráficos acima, podemos observar:

    - A taxa de desemprego se mantém constante nos momentos de pico de vendas.
    - Aonde a o maior deslocamento da taxa de desemprego, as vendas continuam iguais na média.

### H3. Quanto maior o preço do combustível, menor as vendas.

Falso: possuo alto volume de vendas tanto com o valor do combustível mais baixo quanto alto.

In [ ]:
aux1 = df4[['fuel_price', 'weekly_sales']].groupby( 'fuel_price' ).sum().reset_index()

plt.subplot(1,2,1)
bins = list( np.arange( 2, 5, 0.1) )
aux1['fuel_price_binned'] = pd.cut( aux1['fuel_price'],bins=bins )
aux2 = aux1[['fuel_price_binned', 'weekly_sales']].groupby('fuel_price_binned' ).sum().reset_index()
sns.barplot( x='fuel_price_binned', y='weekly_sales', data=aux2 );
plt.xticks( rotation=90 );

plt.subplot(1,2,2)
x = sns.heatmap( aux1.corr( method='pearson' ), annot=True );
bottom, top = x.get_ylim()
x.set_ylim( bottom+0.5, top-0.5 );

In [ ]:
weekly_sales_2010 = df4[df4.year == 2010].groupby('week_of_year')['weekly_sales'].mean()
fuel_price_2010          = df4[df4.year == 2010].groupby('week_of_year')['fuel_price'].mean()

weekly_sales_2011 = df4[df4.year == 2011].groupby('week_of_year')['weekly_sales'].mean()
fuel_price_2011          = df4[df4.year == 2011].groupby('week_of_year')['fuel_price'].mean()

plt.subplot(2,2,1)
plt.title("Média das Vendas Semanais - 2010")
plt.plot(weekly_sales_2010.index, weekly_sales_2010.values, 's-b')

plt.subplot(2,2,3)
plt.title("Preço Médio do Combustível - 2010")
plt.plot(fuel_price_2010.index, fuel_price_2010.values, 'o-r')

plt.subplot(2,2,2)
plt.title("Média das Vendas Semanais - 2011")
plt.plot(weekly_sales_2011.index, weekly_sales_2011.values, 's-b')

plt.subplot(2,2,4)
plt.title("Preço Médio do Combustível - 2011")
plt.plot(fuel_price_2011.index, fuel_price_2011.values, 'o-r')

Podemos observar nos gráfico acima que as vendas se mantem constantes nas primeiras semanas do ano, mesmo com o aumento/queda do preço.

- Nas semanas finais, aonde temos o pico de vendas, mesmo o combustível subindo (2010) e caindo (2011), isso não afeta o número de vendas.

- Baixa correlação inversa entre preço do combustível e vendas semanais (-0.021)

### H4. Quanto maior a temperatura, maior as vendas.

Falso: as vendas se mantém altas com a temperatura na média.

In [ ]:
aux1 = df4[['temperature', 'weekly_sales']].groupby( 'temperature' ).sum().reset_index()

plt.subplot(1,2,1)
bins = list( np.arange( -3, 101, 1) )
aux1['temperature_binned'] = pd.cut( aux1['temperature'],bins=bins )
aux2 = aux1[['temperature_binned', 'weekly_sales']].groupby('temperature_binned' ).sum().reset_index()
sns.barplot( x='temperature_binned', y='weekly_sales', data=aux2 );
plt.xticks( rotation=90 );

plt.subplot(1,2,2)
x = sns.heatmap( aux1.corr( method='pearson' ), annot=True );
bottom, top = x.get_ylim()
x.set_ylim( bottom+0.5, top-0.5 );

In [ ]:
weekly_sales_2010 = df4[df4.year == 2010].groupby('week_of_year')['weekly_sales'].mean()
temperature_2010          = df4[df4.year == 2010].groupby('week_of_year')['temperature'].mean()

weekly_sales_2011 = df4[df4.year == 2011].groupby('week_of_year')['weekly_sales'].mean()
temperature_2011          = df4[df4.year == 2011].groupby('week_of_year')['temperature'].mean()

plt.subplot(2,2,1)
plt.title("Média das Vendas Semanais - 2010")
plt.plot(weekly_sales_2010.index, weekly_sales_2010.values, 's-b')

plt.subplot(2,2,3)
plt.title("Temperatura Média - 2010")
plt.plot(temperature_2010.index, temperature_2010.values, 'o-r')

plt.subplot(2,2,2)
plt.title("Média das Vendas Semanais - 2011")
plt.plot(weekly_sales_2011.index, weekly_sales_2011.values, 's-b')

plt.subplot(2,2,4)
plt.title("Temperatura Média - 2011")
plt.plot(temperature_2011.index, temperature_2011.values, 'o-r')

Podemos verificar nos gráficos acima:

    - A variação de temperatura se mantém muito parecida em ambos os anos.
    
    - Baixa correlação entre temperatura e vendas semanais (0.088)

### H5. Se tenho feriado na semana, minhas vendas são maiores.

Verdadeiro: semanas com feriados faturam mais que semanas normais.

In [ ]:
aux1 = df4[['is_holiday', 'weekly_sales']].groupby( 'is_holiday' ).mean().reset_index()

plt.subplot(1,2,1)
sns.barplot( x='is_holiday', y='weekly_sales', data=aux1 );
plt.xticks( rotation=90 );

plt.subplot(1,2,2)
x = sns.heatmap( aux1.corr( method='pearson' ), annot=True );
bottom, top = x.get_ylim()
x.set_ylim( bottom+0.5, top-0.5 );

Como podemos observar nos gráficos acima, as 2 variáveis estão perfeitamente correlacionadas. Ou seja, o fato de ser semana de feriado impacta diretamente das vendas semanais.

### H6.	Quanto maior a loja, maior as vendas.

Falso: existem lojas de tamanho menor que faturam mais que lojas maiores.

In [ ]:
aux1 = df4[['size', 'weekly_sales']].groupby( 'size' ).mean().reset_index()

plt.subplot(1,2,1)
bins = list( np.arange( 30000, 230000, 10000) )
aux1['size_binned'] = pd.cut( aux1['size'],bins=bins )
aux2 = aux1[['size_binned', 'weekly_sales']].groupby('size_binned' ).sum().reset_index()
sns.barplot( x='size_binned', y='weekly_sales', data=aux2 );
plt.xticks( rotation=90 );

plt.subplot(1,2,2)
x = sns.heatmap( aux1.corr( method='pearson' ), annot=True );
bottom, top = x.get_ylim()
x.set_ylim( bottom+0.5, top-0.5 );



As lojas de tamanho entre 200000 - 210000 são responsáveis pela grande maioria das vendas

O tamanho das lojas possuem alta correlação com as vendas semanais (0.79).

### H7. Verificação das Vendas por Departamento.

In [ ]:
aux1 = df4[['dept', 'weekly_sales']].groupby( 'dept' ).sum().reset_index()

plt.subplot(1,2,1)
sns.barplot( x='weekly_sales', y='dept', data=aux1, orient = 'h' );
plt.xticks( rotation=90 );

plt.subplot(1,2,2)
x = sns.heatmap( aux1.corr( method='pearson' ), annot=True );
bottom, top = x.get_ylim()
x.set_ylim( bottom+0.5, top-0.5 );

In [ ]:
aux1.sort_values('weekly_sales', ascending = True).head()

Os 5 Departamentos que mais venderam são: 92, 95, 38, 72 e 90

Os 5 Departamentos que menos venderam são: 43, 39, 78, 51 e 45

Os Tipos de departamento tem uma pequena correlação com as vendas (0.13).

## 4.3 Análise Multivariada

Nessa seção iremos verificar como as variáveis se relacionam.

### 4.3.1 Atributos Numéricos

In [ ]:
a = df4
a = a.drop(columns = ['type',], axis = 1)
correlation = a.corr(method = 'pearson')
sns.heatmap(correlation, annot = True);

Como podemos observar a baixa correlação entre as promoções e as vendas semanais, podemos optar por não seguir com elas.

# 5.0 Data Preparation

Nessa seção iremos preparar os dados para a utilização dos algoritmos de machine learning. Como temos diversos tipos de dados, de várias escalas diferentes, é interessante trabalhar neles para que fiquem em uma escala parecida, para o algoritmo poder performar melhor.

In [ ]:
df5 = df4.copy()
df5.head()

## 5.1 Normalização

Como não possuimos nenhuma variável que tenha esse comportamento, não iremos utilizar essa transformação.

## 5.2 Rescaling

In [ ]:
#pegando somente as variáveis numéricas
a = df5.select_dtypes(include = ['int64','float64'])
a = a.drop(columns = ['mark_down1','mark_down2','mark_down3','mark_down4','mark_down5','is_holiday'], axis = 1)
a.head()

In [ ]:
#verificação dos outliers das variáveis numéricas

plt.subplot( 7, 1, 1 )
sns.boxplot(df5['weekly_sales']);

plt.subplot( 7, 1, 2 )
sns.boxplot(df5['temperature']);

plt.subplot( 7, 1, 3 )
sns.boxplot(df5['fuel_price']);

plt.subplot( 7, 1, 4 )
sns.boxplot(df5['cpi']);

plt.subplot( 7, 1, 5 )
sns.boxplot(df5['unemployment']);

plt.subplot( 7, 1, 6 )
sns.boxplot(df5['size']);

Por meio do gráfico acima, podemos verificar que as features que tem mais outliers são: 'temperature' e 'weekly_sales'.

In [ ]:
rs = RobustScaler() #quando a feature tem muitos outliers
mms = MinMaxScaler()

df5['temperature'] = rs.fit_transform( df5[['temperature']].values )

df5['fuel_price'] = mms.fit_transform( df5[['fuel_price']].values )

df5['cpi'] = mms.fit_transform( df5[['cpi']].values )

df5['unemployment'] = mms.fit_transform( df5[['unemployment']].values )

df5['size'] = mms.fit_transform( df5[['size']].values )


Após a transformação, podemos ver a nova distribuição das variáveis abaixo:

In [ ]:
plt.subplot( 7, 1, 1 )
sns.distplot(df5['temperature']);

plt.subplot( 7, 1, 2 )
sns.distplot(df5['fuel_price']);

plt.subplot( 7, 1, 3 )
sns.distplot(df5['cpi']);

plt.subplot( 7, 1, 4 )
sns.distplot(df5['unemployment']);

plt.subplot( 7, 1, 5 )
sns.distplot(df5['size']);

## 5.3 Transformação

### 5.3.1 Encoding

Variáveis que não são numéricas, como o tipo das lojas, devemos utilizar uma transformação para torna-las numéricas.
O encoding escolhido para a variável 'type', será o 'label encoding'.

In [ ]:
a = df5.select_dtypes(include = 'object')
a = a.drop(columns = ['year_week'], axis = 1)
a.head()

In [ ]:
le = LabelEncoder() #trocou as letras por números
df5['type'] = le.fit_transform(df5['type'])

### 5.3.2 Response Variable Transformation

In [ ]:
df5['weekly_sales'] = np.log1p(df5['weekly_sales'])

In [ ]:
sns.distplot(df5['weekly_sales'])

Agora temos uma variável resposta com um comportamento se 'aproximando' de uma normal.

### 5.3.3 Nature Transformation

In [ ]:
# variáveis cíclicas:

# month
df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x * (2 * np.pi / 12)))
df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x * (2 * np.pi / 12)))

# day
df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x * (2 * np.pi / 30)))
df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x * (2 * np.pi / 30)))

# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x: np.sin(x * (2 * np.pi / 52)))
df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x: np.cos(x * (2 * np.pi / 52)))


# 6.0 Featuring Selection

Nessa seção iremos avaliar quais são as features mais importantes para trabalharmos no modelo.

In [ ]:
df6 = df5.copy()

## 6.1 Split dataframe into training and test Dataset

Aqui iremos deletar as variáveis que deram origem as outras diferentes na seção passada.

In [ ]:
cols_drop = ['month', 'day', 'week_of_year','year_week','mark_down1','mark_down2','mark_down3',
            'mark_down4','mark_down5']
df6 = df6.drop(cols_drop, axis = 1)

# 7.0 Machine Learning Modeling

In [ ]:
df7 = df6.copy()

In [ ]:
X = df7.drop(columns = ['date','weekly_sales'])
X.head()

In [ ]:
y = df7['weekly_sales']

## 7.0.1 Tratamento dos dados de teste

Agora precisamos trazer os dados de teste e aplicar todas as transformações que utilizamos nos dados de treino para que possamos utilizar ambos nos algoritmos de machine learning.

In [ ]:
df_raw_test.head()

In [ ]:
df11 = df_raw_test.copy()

In [ ]:
# padronizando os nomes das colunas
cols_old = ['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature',
'Fuel_Price', 'MarkDown1', 'MarkDown2',
'MarkDown3', 'MarkDown4', 'MarkDown5',
'CPI','Unemployment', 'Type', 'Size']
snakecase = lambda x: inflection.underscore( x )
cols_new = list( map( snakecase, cols_old ) )
# rename
df11.columns = cols_new

In [ ]:
df11['date'] = pd.to_datetime( df11['date'] )

df11['is_holiday'] = df11['is_holiday'].astype('int64')

In [ ]:
df11.isna().sum()

Como verificamos nas outras seções que o 'cpi' e o 'unemployment' não afetam tanto as vendas, iremos preencher com valores nulos aonde estão como NaN. E não iremos considerar as promoções também.

In [ ]:
df11.fillna(0, inplace = True)
df11.head()

In [ ]:
df11.isna().sum()

In [ ]:
df21 = df11.copy()

In [ ]:
# year
df21['year'] = df21['date'].dt.year

# month
df21['month'] = df21['date'].dt.month

# day
df21['day'] = df21['date'].dt.day

# week of year
df21['week_of_year'] = df21['date'].dt.weekofyear

# year week
df21['year_week'] = df21['date'].dt.strftime( '%Y-%W' )

In [ ]:
df51 = df21.copy()

In [ ]:
df51['temperature'] = rs.fit_transform( df51[['temperature']].values )

df51['fuel_price'] = mms.fit_transform( df51[['fuel_price']].values )

df51['cpi'] = mms.fit_transform( df51[['cpi']].values )

df51['unemployment'] = mms.fit_transform( df51[['unemployment']].values )

df51['size'] = mms.fit_transform( df51[['size']].values )

df51['type'] = le.fit_transform(df51['type'])

# variáveis cíclicas:

# month
df51['month_sin'] = df51['month'].apply(lambda x: np.sin(x * (2 * np.pi / 12)))
df51['month_cos'] = df51['month'].apply(lambda x: np.cos(x * (2 * np.pi / 12)))

# day
df51['day_sin'] = df51['day'].apply(lambda x: np.sin(x * (2 * np.pi / 30)))
df51['day_cos'] = df51['day'].apply(lambda x: np.cos(x * (2 * np.pi / 30)))

# week of year
df51['week_of_year_sin'] = df51['week_of_year'].apply(lambda x: np.sin(x * (2 * np.pi / 52)))
df51['week_of_year_cos'] = df51['week_of_year'].apply(lambda x: np.cos(x * (2 * np.pi / 52)))

In [ ]:
x_tester = df51.copy()

In [ ]:
cols_drop = ['date','month', 'day', 'week_of_year','year_week','mark_down1','mark_down2','mark_down3',
            'mark_down4','mark_down5']
x_tester = x_tester.drop(cols_drop, axis = 1)

Agora iremos verificar qual o modelo tenha a melhor performance em nossos dados.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

## 7.1 Linear Regression Model

In [ ]:
#model
lr = LinearRegression().fit(X_train, y_train)

#prediction
yhat_lr = lr.predict(X_test)

In [ ]:
print(lr.score(X_train, y_train))
print(lr.score(X_test,y_test))

## 7.2 Linear Regression Regularized Model - Lasso

In [ ]:
#model
lrr = Lasso(alpha = 0.01).fit(X_train, y_train)

#prediction
yhat_lrr = lrr.predict(X_test)

In [ ]:
print(lrr.score(X_train, y_train))
print(lrr.score(X_test,y_test))

Observem com os resultados acima, que modelos lineares não conseguem performar bem em cima da nossa base de dados.

## 7.3 Random Forest Regressor

In [ ]:
#model
rf = RandomForestRegressor(n_estimators = 100, n_jobs = -1, random_state = 42).fit(X_train, y_train)

#prediction
yhat_rf = rf.predict(X_test)

In [ ]:
print(rf.score(X_train, y_train))
print(rf.score(X_test,y_test))

## 7.4 XGBoost Regressor

In [ ]:
#model
model_xgb = xgb.XGBRegressor().fit(X_train, y_train )

#prediction
yhat_xgb = model_xgb.predict(X_test)

In [ ]:
print(model_xgb.score(X_train, y_train))
print(model_xgb.score(X_test,y_test))

O modelo com a acurácia mais alta apresentada foi 'Random Forest Regressor', então iremos seguir com ele.

# Insight's

A seguir seguem alguns insight's que podemos tirar do estudo de caso:

    Insight:
      - O tamanho da loja tem um impacto grande no número de vendas de algumas lojas.
      Acionável:
      - Expansão de lojas de menores pode ser uma boa estratégia para aumento do lucro no longo prazo.
      
      Insight:
      - A variação de indicadores econômicos não tem tanto impacto nas vendas.
      Acionável:
      - Olhar mais para os KPI's do negócio do que KPI's macro.
      
      Insight:
      - Picos de vendas sempre acontecem nos últimos meses do ano.
      Acionável:
      - Prever no budget esse turnover de funcionários e quantidade de estoque dos produtos demandados.
      
      Insight:
      - As vendas se mantém alta com a temperatura média.
      Acionável:
      - Não ter tanto foco em produtos que se destinam a casos mais extremos de temperatura.
      
      
      
      

## Próximos Passos:

    - Aplicar o Cross-Validation nos modelos treinados
    - Ajuste fino de hiperparâmetros do modelo escolhido
    - Modelagem das promoções nos anos que temos disponíveis